In [2]:
%load_ext autoreload
%autoreload 2

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
import os
from tqdm import tqdm
import glob
import csv
from typing import Any

from cs336_alignment.zeroshot import parse_mmlu_response

# Load Qwen model and tokenizer
model_path = "../models/Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, torch_dtype=torch.float16, device_map="auto")

# MMLU test folder
mmlu_test_dir = "../data/mmlu/test"
csv_files = glob.glob(os.path.join(mmlu_test_dir, "*.csv"))

# Format input prompts
def format_prompt(example):
    prompt = f"Question: {example['question']}\n"
    for i, choice in enumerate(example['choices']):
        prompt += f"({chr(65+i)}) {choice}  "
    prompt += "\nAnswer:"
    return prompt

# Generate predictions
def generate_answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=10)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded

# Evaluate each subject
os.makedirs("evaluation_results", exist_ok=True)

for file_path in csv_files:
    category = os.path.splitext(os.path.basename(file_path))[0]
    examples = []

    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            examples.append({
                "question": row[0],
                "choices": row[1:5],
                "answer": row[5].strip().upper(),
                "subject": category
            })

    results = []
    correct = 0

    for example in tqdm(examples, desc=f"Evaluating {category}"):
        prompt = format_prompt(example)
        # break
        raw_output = generate_answer(prompt)
        pred = parse_mmlu_response(example, raw_output)
        gt = example['answer']
        results.append({
            "question": example['question'],
            "choices": example['choices'],
            "ground_truth": gt,
            "prediction": pred,
            "raw_output": raw_output
        })
        if pred == gt:
            correct += 1

    accuracy = correct / len(results)
    print(f"Zero-shot accuracy on {category}: {accuracy:.2%}")
    with open(f"evaluation_results/qwen_mmlu_{category}.json", "w") as f:
        json.dump({"accuracy": accuracy, "results": results}, f, indent=2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Evaluating high_school_government_and_politics_test:   0%|          | 0/193 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating high_school_government_and_politics_test:   1%|          | 1/193 [00:00<00:22,  8.68it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating high_school_government_and_politics_test:   1%|          | 2/193 [00:00<00:21,  8.72it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating high_school_government_and_politics_test:   2%|▏         | 4/193 [00:00<00:27,  6.83it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating high_school_government_and_politics_test:   3%|▎         | 5/193 [00:00<00:25,  7.37it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating high_school_government_and_politics_test:   3%|▎         | 

Zero-shot accuracy on high_school_government_and_politics_test: 53.37%


Evaluating security_studies_test:   2%|▏         | 6/245 [00:01<00:39,  6.11it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating security_studies_test:  25%|██▍       | 61/245 [00:11<00:34,  5.35it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating security_studies_test:  33%|███▎      | 80/245 [00:14<00:33,  4.91it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating security_studies_test:  48%|████▊     | 118/245 [00:20<00:18,  7.06it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating security_studies_test:  51%|█████▏    | 126/245 [00:22<00:20,  5.91it/s]Setti

Zero-shot accuracy on security_studies_test: 45.71%


...

Zero-shot accuracy on high_school_geography_test: 55.05%


Evaluating us_foreign_policy_test:   5%|▌         | 5/100 [00:00<00:11,  8.15it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating us_foreign_policy_test:  10%|█         | 10/100 [00:01<00:13,  6.49it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating us_foreign_policy_test:  31%|███       | 31/100 [00:04<00:09,  6.99it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating us_foreign_policy_test:  33%|███▎      | 33/100 [00:05<00:06,  9.74it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating us_foreign_policy_test:  41%|████      | 41/100 [00:06<00:09,  6.00it/s]Se

Zero-shot accuracy on us_foreign_policy_test: 68.00%


Evaluating prehistory_test:  31%|███       | 101/324 [00:18<00:35,  6.28it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating prehistory_test:  56%|█████▌    | 181/324 [00:30<00:21,  6.51it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating prehistory_test:  75%|███████▌  | 244/324 [00:41<00:10,  7.47it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating prehistory_test:  80%|███████▉  | 258/324 [00:43<00:13,  4.98it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Evaluating prehistory_test:  83%|████████▎ | 270/324 [00:45<00:09,  5.81it/s]Setting `pad_token_id` to `eos_

Zero-shot accuracy on prehistory_test: 48.77%


In [5]:
import glob
import csv
import json
import os

def summarize_evaluation_results(results_dir="evaluation_results"):
    total_correct = 0
    total_questions = 0
    total_none_outputs = 0

    result_files = glob.glob(os.path.join(results_dir, "qwen_mmlu_*.json"))
    
    for file_path in result_files:
        with open(file_path, "r") as f:
            data = json.load(f)
            results = data["results"]
            for result in results:
                pred = result["prediction"]
                gt = result["ground_truth"]
                if pred == "None" or pred == "none" or pred == None: 
                    total_none_outputs += 1
                if pred == gt:
                    total_correct += 1
                total_questions += 1

    overall_accuracy = total_correct / total_questions if total_questions > 0 else 0.0
    print(f"Overall accuracy across all subjects: {overall_accuracy:.2%}")
    print(f"Total predictions: {total_questions}")
    print(f"Total 'None' predictions: {total_none_outputs}")
summarize_evaluation_results()

Overall accuracy across all subjects: 42.89%
Total predictions: 14042
Total 'None' predictions: 0
